# VencoPy Tutorial 2

This tutorial aims to give a more in depth overview into the DataParser class and showcases some features that can be customised.

In [1]:
import sys
import os
from os import path
from pathlib import Path

sys.path.append((path.dirname(path.dirname(Path.cwd()))))

from vencopy.core.dataparsers import parse_data
from vencopy.utils.utils import load_configs, create_output_folders

print("Current working directory: {0}".format(os.getcwd()))

Current working directory: c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\tutorials


In [2]:
base_path = Path.cwd().parent
configs = load_configs(base_path)
create_output_folders(configs=configs)
    
# Adapt relative paths in config for tutorials
configs['dev_config']['global']['relative_path']['parse_output'] = Path.cwd().parent.parent / configs['dev_config']['global']['relative_path']['parse_output']
configs['dev_config']['global']['relative_path']['diary_output'] = Path.cwd().parent.parent / configs['dev_config']['global']['relative_path']['diary_output']
configs['dev_config']['global']['relative_path']['grid_output'] = Path.cwd().parent.parent / configs['dev_config']['global']['relative_path']['grid_output']
configs['dev_config']['global']['relative_path']['flex_output'] = Path.cwd().parent.parent / configs['dev_config']['global']['relative_path']['flex_output']
configs['dev_config']['global']['relative_path']['aggregator_output'] = Path.cwd().parent.parent / configs['dev_config']['global']['relative_path']['aggregator_output']
configs['dev_config']['global']['relative_path']['processor_output'] = Path.cwd().parent.parent / configs['dev_config']['global']['relative_path']['processor_output']

# Set reference dataset
datasetID = 'MiD17'

# Modify the localPathConfig file to point to the .csv file in the sampling folder in the tutorials directory where the dataset for the tutorials lies.
configs['user_config']['global']['absolute_path'][datasetID] = Path.cwd() /'data_sampling'

# Similarly we modify the datasetID in the global config file
configs['dev_config']['global']['files'][datasetID]['trips_data_raw'] = datasetID + '.csv'

# We also modify the parseConfig by removing some of the columns that are normally parsed from the MiD, which are not available in our semplified test dataframe
del configs['dev_config']['dataparsers']['data_variables']['household_id']
del configs['dev_config']['dataparsers']['data_variables']['person_id']


## DataParser config file

The DataParser config file defines which variables are to be parsed (i.e. the ones needed to create trip diaries and calculate fleet flexibility) and sets some filtering options, such as the conditions for trips to be included of excluded from the parsing.

<div class="alert alert-block alert-danger"><b>Warning:</b> The list is very long.</div>

In [3]:
configs['dev_config']

{'global': {'relative_path': {'parse_output': WindowsPath('c:/Users/mior_fa/Documents/7_Work/vencopy/vencopy_internal/vencopy/output/dataparser'),
   'diary_output': WindowsPath('c:/Users/mior_fa/Documents/7_Work/vencopy/vencopy_internal/vencopy/output/diarybuilder'),
   'grid_output': WindowsPath('c:/Users/mior_fa/Documents/7_Work/vencopy/vencopy_internal/vencopy/output/gridmodeller'),
   'flex_output': WindowsPath('c:/Users/mior_fa/Documents/7_Work/vencopy/vencopy_internal/vencopy/output/flexestimator'),
   'aggregator_output': WindowsPath('c:/Users/mior_fa/Documents/7_Work/vencopy/vencopy_internal/vencopy/output/profileaggregator'),
   'processor_output': WindowsPath('c:/Users/mior_fa/Documents/7_Work/vencopy/vencopy_internal/vencopy/output/postprocessor'),
   'config': './config/'},
  'files': {'MiD17': {'enrypted_zip_file_B1': 'B1_Standard-DatensatzpaketEncrypted.zip',
    'enrypted_zip_file_B2': 'B2_Regional-DatensatzpaketEncrypted.zip',
    'households_data_raw': 'MiD2017_Region

## _DataParser_ class

Let's first run the class and see the outputs we get.

In [4]:
data = parse_data(configs=configs)
data.process()

Generic file parsing properties set up.
Starting to retrieve local data file from c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\tutorials\data_sampling\MiD17.csv.
Finished loading 2124 rows of raw data of type .csv.
Running in debug mode.
Finished harmonization of variables.
Finished harmonization of ID variables.
Starting filtering, applying 8 filters.
All filters combined yielded that a total of 241 trips are taken into account.
This corresponds to 40.166666666666664 percent of the original data.
Completed park timestamp adjustments.
Finished activity composition with 240 trips and 240 parking activites.


OSError: Cannot save file into a non-existent directory: 'c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\output\dataparser'

We can see from the print statements in the class that after reading in the initial dataset, which contained 2124 rows, and applying 8 filters, we end up with a database containing 1980 suitable entries, which corresponds to about 93% of the initial sample.
These trip respect the condition that they all need to be shorter than 1000km, which is set in the parseConfig under the 'filterDict' key.

Now we can, for example, change in the filters the maximum allowed trip distance from 1000km to 50km and see how this affects the resulting available trips (the extreme case of 50km is only used for the tutorial purpose).

In [5]:
configs['dev_config']['dataparsers']['filters'][datasetID]['smaller_than']['trip_distance'] = [50]

In [6]:
data = parse_data(configs=configs)
data.process()

Generic file parsing properties set up.
Starting to retrieve local data file from c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\tutorials\data_sampling\MiD17.csv.
Finished loading 2124 rows of raw data of type .csv.
Running in debug mode.
Finished harmonization of variables.
Finished harmonization of ID variables.
Starting filtering, applying 8 filters.
All filters combined yielded that a total of 233 trips are taken into account.
This corresponds to 38.83333333333333 percent of the original data.
Completed park timestamp adjustments.
Finished activity composition with 232 trips and 232 parking activites.


OSError: Cannot save file into a non-existent directory: 'c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\output\dataparser'

We can see how with a maximum trip distance of 1000km, all filters combined yielded a total of 1980 trips, which corresponds to about 93% of the original dataset. By changing this values to 50km, additional 82 trips have been excluded, resulting in 1898 trips (89% ofthe initial dataset).

## Next Steps

In the next tutorial, you will learn more in detail the internal workings of the FlexEstimator class and how to customise some settings.